In [6]:
import numpy as np
import tensorflow as tf
tf.__version__
import pandas as pd

In [7]:
# chargement du fichier
doc = pd.read_table('fra.txt', header=None, sep='\t')
doc.head()

,0,1
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Wow!,Ça alors !


In [8]:
#prenons simplement un sample de 5000 phrases pour eviter les lenteurs
doc = doc.sample(5000)

In [9]:
# chargement des languages francais et anglais de spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 47.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# import de chacun des languages
import fr_core_news_sm
import en_core_web_sm
nlp_fr = fr_core_news_sm.load()
nlp_en = en_core_web_sm.load()



In [11]:
# chargement du corpus entier de phrases françaises et anglaises
fr_corpus = "".join(doc.iloc[:,1].to_list())
en_corpus = "".join(doc.iloc[:,0].to_list())

In [12]:
fr_corpus

'Je me sens mal à ce sujet.Ne me rappelle plus jamais !Ne parlons pas de ça devant les enfants.Etes-vous sûr que vous ne voulez pas vivre à la maison avec vos parents ?Je croyais que elle était enceinte.Je suis surprise de te voir.Il faut que tu sois plus patient.Je me rappelle l\'avoir rencontrée quelque part.Il n\'est pas en ville.Ne me laissez pas dans l\'ignorance à ce sujet.J\'ai honte de la paresse de mon fils.Cessez de pleurer comme une fillette !J\'ai perdu mon portefeuille quelque part par ici.Je n\'arrive pas à détecter de quelconque récurrence.Elles sont intactes.Tu es la maîtresse.Il dit qu\'il s\'est rendu au Japon la semaine précédente.Moi non plus, je n\'aime pas ça.Je ne veux pas me perdre en détails.Vous ne m\'écoutez pas.Je serai toujours avec vous.Ça n\'empire pas.Je ne peux rien faire pour Tom.Je voulais juste te dire merci.Nous ne sommes pas autorisés à faire cela.Qui a fait du bénévolat ?Tu as déjà mangé de la tarte à la banane\u202f?Il a raté son vol.Je l\'ai vue

In [13]:
# chargement des deux corpus dans spacy
# %%time
import time
nlp_fr.max_length = len(fr_corpus)
nlp_en.max_length =len(en_corpus)

fr_doc = nlp_fr(fr_corpus)
en_doc = nlp_en(en_corpus )

In [14]:
# Tokenisation de chacune des phrases via spacy
#%% time
doc["fr_tokens"] = doc.iloc[:,1].apply( lambda x: nlp_fr.tokenizer(x))
doc["en_tokens"] = doc.iloc[:,0].apply(lambda x: nlp_en.tokenizer(x))

In [15]:
doc.tail()

,0,1,fr_tokens,en_tokens
126687,Turn left at the second traffic light.,Tournez au second feu à gauche !,"(Tournez, au, second, feu, à, gauche, !)","(Turn, left, at, the, second, traffic, light, .)"
41689,We don't want to wait.,Nous ne voulons pas attendre.,"(Nous, ne, voulons, pas, attendre, .)","(We, do, n't, want, to, wait, .)"
58381,Maybe I should take this.,Peut-être devrais-je prendre ceci.,"(Peut, -, être, devrais, -je, prendre, ceci, .)","(Maybe, I, should, take, this, .)"
30630,Why are you so busy?,Pourquoi êtes-vous si occupées ?,"(Pourquoi, êtes, -vous, si, occupées, ?)","(Why, are, you, so, busy, ?)"
158845,It was just 6 years ago that I visited New Zea...,Cela fait 6 ans exactement que je suis allé en...,"(Cela, fait, 6, ans, exactement, que, je, suis...","(It, was, just, 6, years, ago, that, I, visite..."


In [16]:
# creation d'un set qui va prendre tous les tokens unique de notre code
en_tokens = [token.text for token in en_doc]
en_vocabulary_set = set(en_tokens) # retirer les doublons
en_vocab_size = len(en_vocabulary_set)
print(en_vocab_size)

4310


In [17]:
# meme chose pour le français
fr_tokens = [token.text for token in en_doc]
fr_vocabulary_set = set(fr_tokens) #retirer les doublons
fr_vocab_size = len(fr_vocabulary_set)
print(fr_vocab_size)

4310


In [18]:
# creation d'un id pour chacun des tokens
all_en_tokens = {}
for i, en_token in enumerate(en_vocabulary_set):
    all_en_tokens[en_token] = i+1 #on prend à i+1 pour laisser la valeur

all_fr_tokens = {}
for i, fr_token in enumerate(fr_vocabulary_set):
  all_fr_tokens[fr_token] = i+1

In [19]:
def en_tokens_to_index(tokens):
    indices = []
    for token in tokens:
        if token.text in all_en_tokens:
            indices.append(all_en_tokens[token.text])
        else:
            print("Token '{}' not found in dictionary.".format(token.text))
    return indices

def fr_tokens_to_index(tokens):
    indices = []
    for token in tokens:
        if token.text in all_fr_tokens:
            indices.append(all_fr_tokens[token.text])
        else:
            print("Token '{}' not found in dictionary.".format(token.text))
    return indices


In [20]:
# transformation des tokens en indices
doc["fr_indices"] = doc['fr_tokens'].apply(lambda x: fr_tokens_to_index(x))
doc["en_indices"] = doc['en_tokens'].apply(lambda x: en_tokens_to_index(x))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Token 'ne' not found in dictionary.
Token 'peut' not found in dictionary.
Token 'pas' not found in dictionary.
Token 'te' not found in dictionary.
Token 'parler' not found in dictionary.
Token 'Je' not found in dictionary.
Token 'suis' not found in dictionary.
Token 'garé' not found in dictionary.
Token 'au' not found in dictionary.
Token 'de' not found in dictionary.
Token 'la' not found in dictionary.
Token 'rue' not found in dictionary.
Token 'Il' not found in dictionary.
Token 'craint' not found in dictionary.
Token 'la' not found in dictionary.
Token 'mort' not found in dictionary.
Token 'L'' not found in dictionary.
Token 'Allemagne' not found in dictionary.
Token 'fut' not found in dictionary.
Token 'autrefois' not found in dictionary.
Token 'l'' not found in dictionary.
Token 'alliée' not found in dictionary.
Token 'de' not found in dictionary.
Token 'l'' not found in dictionary.
Token 'Italie' not fo

In [21]:
doc.tail

<bound method NDFrame.tail of                                                         0  \
38598                              I feel bad about that.   
55453                           Don't ever call me again.   
146874   Let's not discuss this in front of the children.   
156750  Are you sure you don't want to live at home wi...   
99972                    I thought that she was pregnant.   
...                                                   ...   
126687             Turn left at the second traffic light.   
41689                              We don't want to wait.   
58381                           Maybe I should take this.   
30630                                Why are you so busy?   
158845  It was just 6 years ago that I visited New Zea...   

                                                        1  \
38598                          Je me sens mal à ce sujet.   
55453                        Ne me rappelle plus jamais !   
146874           Ne parlons pas de ça devant les enfants.   
156750  Etes-vous sûr que vous ne voulez pas vivre à l...   
99972                 Je croyais que elle était enceinte.   
...                                                   ...   
126687                   Tournez au second feu à gauche !   
41689                       Nous ne voulons pas attendre.   
58381                  Peut-être devrais-je prendre ceci.   
30630                    Pourquoi êtes-vous si occupées ?   
158845  Cela fait 6 ans exactement que je suis allé en...   

                                                fr_tokens  \
38598                (Je, me, sens, mal, à, ce, sujet, .)   
55453                 (Ne, me, rappelle, plus, jamais, !)   
146874  (Ne, parlons, pas, de, ça, devant, les, enfant...   
156750  (Etes, -vous, sûr, que, vous, ne, voulez, pas,...   
99972        (Je, croyais, que, elle, était, enceinte, .)   
...                                                   ...   
126687           (Tournez, au, second, feu, à, gauche, !)   
41689               (Nous, ne, voulons, pas, attendre, .)   
58381     (Peut, -, être, devrais, -je, prendre, ceci, .)   
30630            (Pourquoi, êtes, -vous, si, occupées, ?)   
158845  (Cela, fait, 6, ans, exactement, que, je, suis...   

                                                en_tokens  \
38598                      (I, feel, bad, about, that, .)   
55453                 (Do, n't, ever, call, me, again, .)   
146874  (Let, 's, not, discuss, this, in, front, of, t...   
156750  (Are, you, sure, you, do, n't, want, to, live,...   
99972           (I, thought, that, she, was, pregnant, .)   
...                                                   ...   
126687   (Turn, left, at, the, second, traffic, light, .)   
41689                    (We, do, n't, want, to, wait, .)   
58381                   (Maybe, I, should, take, this, .)   
30630                        (Why, are, you, so, busy, ?)   
158845  (It, was, just, 6, years, ago, that, I, visite...   

                      fr_indices  \
38598               [2800, 4040]   
55453               [2800, 1823]   
146874                    [4040]   
156750              [1692, 2768]   
99972                     [4040]   
...                          ...   
126687              [2575, 1823]   
41689                     [4040]   
58381               [1030, 4040]   
30630                     [2768]   
158845  [3396, 3132, 1030, 4040]   

                                               en_indices  
38598                 [1378, 269, 3847, 2386, 2862, 4040]  
55453            [1303, 778, 4074, 2337, 2800, 435, 4040]  
146874  [2770, 9, 1355, 3726, 1504, 3035, 2940, 3985, ...  
156750  [2277, 3242, 3460, 3242, 3042, 778, 2437, 3391...  
99972           [1378, 479, 2862, 2484, 3003, 2929, 4040]  
...                                                   ...  
126687   [4029, 4234, 2038, 2883, 2575, 1733, 2898, 4040]  
41689           [3280, 3042, 778, 2437, 3391, 2309, 4040]  
58381                [2691, 1378, 2381, 1181, 1504, 4040]  
30630

In [22]:
#creation d'une fonction qui va compter la longeur maximum d'une phrase
def max_len(lines):
    return max(len(line) for line in lines)


In [23]:
#application de la fonction sur les tokens francais et angalais
fr_max_len=max_len(doc['fr_indices'].to_list())
en_max_len=max_len(doc['en_indices'].to_list())

In [24]:
tf.keras.preprocessing.sequence.pad_sequences?

In [25]:
#Utilisation de Keras pour creer des sequences de tokens de la meme longueur
# %%time
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["fr_indices"],
                                                                maxlen=fr_max_len,
                                                                padding = "post")
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"],
                                                                maxlen=en_max_len,
                                                                padding = "post")

In [26]:
# visualisation de la shape d'un des tenseurs
padded_fr_indices.shape

(5000, 9)

In [27]:
# application de la categorisation de la variable cible
binarized_en_indices = tf.keras.utils.to_categorical(padded_en_indices, num_classes=en_vocab_size +1)
binarized_en_indices

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [28]:
tf.data.dataset.from_tensor_slices?

Object `tf.data.dataset.from_tensor_slices` not found.


In [29]:
# creation de tf.Dtaset pour chacun des tenseurs francais et anglais
fr_ds= tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [30]:
# creation de tf.data.dataset pour chacun des tenseurs francaisa et anglais
tf_ds = tf.data.Dataset.zip((fr_ds, en_ds))

In [31]:
next(iter(tf_ds))

(<tf.Tensor: shape=(9,), dtype=int32, numpy=array([2800, 4040,    0,    0,    0,    0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(30, 4311), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [32]:
# shuffle & Batch
BATCH_SIZE=32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)


In [33]:
len(doc)/32*0.7

109.375

In [34]:
# train test split
TAKE_SIZE = int(0.7*len(doc)/BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

In [35]:
tf.keras.layers.Embedding?

In [36]:
# Creation du modele
model = tf.keras.Sequential([
    # Couche d'Input Word Embedding
    tf.keras.layers.Embedding(fr_vocab_size + 1, 64, mask_zero = True),

    # Couche LSTM Bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),

    # Nouvelle Couche LSTM Bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = False)),

    # Repeat Vector
    tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

    # Nouvelle Couche LSTM
    tf.keras.layers.LSTM(32, return_sequences = True),
    #couche de sortie avec le nombre de neurones en sortie egale au nombre de classe avec focntion softmax
     tf.keras.layers.TimeDistributed( tf.keras.layers.Dense(en_vocab_size+1, activation = "softmax"))
    ])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          275904    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 repeat_vector (RepeatVecto  (None, 30, 128)           0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 30, 32)            20608     
                                                        

In [38]:
# Prediction 'aleatoire' pour tester notre modele
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 9)
1/1 [==============================] - 5s 5s/step
(32, 30, 4311)
(32, 30, 4311)


In [ ]:
# creons un learning rate schedule pour decroitre le learning rate à mesure que nous entrainons
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1090,
    decay_rate=0.96
    staircase=True
)

# utilisation d'un compileur simple avec un optimiseur Adam pour le calcul de nos gradients
optimizer=tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

# Application du modele sur 200 epochs
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=200)

Epoch 1/200
109/109 [==============================] - 58s 337ms/step - loss: 5.3641 - categorical_accuracy: 0.7416 - val_loss: 2.4781 - val_categorical_accuracy: 0.7441
Epoch 2/200


In [ ]:
# test de traduction
for input_text, translation in test_data.take(1):
   fr_text = input_text
   pred = model.predict_classes(input_text)
   translation = translation


In [ ]:

#phrase en français
for indice in fr_text[0]:
  for key, value in all_fr_tokens.items():
    if indice == value:
      print(key)

In [ ]:
for indice in np.argmax( translation, axis=-1)[0]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)

In [ ]:
# phrase traduite en anglais par le modele
for indice in pred[0]:
  for key, value in all_en_tokens.items():
    if indice == value:
      print(key)